In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

#workaround for Google maps
from IPython.display import HTML
from ipywidgets.embed import embed_minimal_html


In [2]:
#Get city data from cities csv file
city_df = pd.read_csv('../output_data/cities.csv')
city_df = city_df.dropna()
city_df.head()

,City,Country,Lat,Lng,Date,Max Temp (F),Cloudiness,Humidity,Wind Speed
0,loei,th,17.42,101.50,1580007262,79.00,70,60,1.12
1,saint-philippe,re,-21.36,55.77,1580007262,73.40,90,83,9.17
2,kapaa,us,22.08,-159.32,1580007263,78.80,40,65,6.93
3,inta,ru,66.03,60.17,1580007263,-9.24,87,99,2.06
4,mataura,pf,-46.19,168.86,1580007263,77.65,64,58,0.58


In [3]:
# Store latitude and longitude in locations
locations = city_df[['Lat', 'Lng']]

# Convert Humidity to float
humidity = city_df['Humidity'].astype(float)


In [4]:
#Configure gmaps
gmaps.configure(api_key = g_key)
centerv= ["0","0"]

# Plot Heatmap
fig = gmaps.figure(map_type="HYBRID")

# Create heat layer
fig = gmaps.figure(zoom_level = 2,center = centerv)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)

# Add layer
fig.add_layer(heat_layer)

# Display map
fig


Figure(layout=FigureLayout(height='420px'))

In [5]:
#Filter cities by weather conditions
ideal_city_df = city_df.copy()


ideal_city_df = ideal_city_df.loc[ideal_city_df['Max Temp (F)'] > 75] 
ideal_city_df = ideal_city_df.loc[ideal_city_df['Wind Speed'] < 10, :]
ideal_city_df = ideal_city_df.loc[ideal_city_df['Cloudiness'] < 40, :]


In [9]:
hotel_df = ideal_city_df.copy()

#hotel_df['Hotel Name'] = ''
target_radius = 5000
target_type = 'hotel'

url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

params = {
    'radius': target_radius,
    'type': target_type,
    'key': g_key
}

for index, row in hotel_df.iterrows():
    
    hotel_df['Hotel Location'] = hotel_df['Lat'].astype(str).str.cat(hotel_df['Lng'].values.astype(str), sep = ',')
    
    params['location'] = hotel_df.loc[index, 'Hotel Location'] 
    
    response = requests.get(url, params = params).json()
    
    results = response['results']
    
    try:
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        hotel_df.loc[index, 'Hotel Address'] = results[-1]['vicinity']
        hotel_df.loc[index, 'Hotel Lat'] = results[0]['geometry']['location']['lat']
        hotel_df.loc[index, 'Hotel Lng'] = results[0]['geometry']['location']['lng']
    
    except (KeyError, IndexError):
        print(f'Missing hotels... skipping.')
        


Missing hotels... skipping.
Missing hotels... skipping.
Missing hotels... skipping.


In [19]:
hotel_df.count()

City              36
Country           36
Lat               36
Lng               36
Date              36
Max Temp (F)      36
Cloudiness        36
Humidity          36
Wind Speed        36
Hotel Location    36
Hotel Name        36
Hotel Address     36
Hotel Lat         36
Hotel Lng         36
dtype: int64

In [24]:
#Drop NaN values
hotel_df = hotel_df.dropna()


# Using the template add the hotel marks to the heatmap
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[['Hotel Lat', 'Hotel Lng']]

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)

fig

Figure(layout=FigureLayout(height='420px'))